In [1]:
import os 
os.chdir('../../')

In [2]:
from backbones.sana import SANA

# 사용 예
model = SANA()
print(model)

/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  4.57it/s]


In [3]:
import matplotlib.pyplot as plt

def show_compare(euler_img, dpm_img, unipc_img):
    fig, axes = plt.subplots(1, 3, figsize=(18, 10))
    axes[0].imshow(euler_img); axes[0].axis('off'); axes[0].set_title('Euler')
    axes[1].imshow(dpm_img);  axes[1].axis('off'); axes[1].set_title('DPM-Solver')
    axes[2].imshow(unipc_img);  axes[2].axis('off'); axes[2].set_title('Dual Log DPM-Solver')
    plt.tight_layout()
    plt.show()


In [4]:
import numpy as np

prompts = np.load('prompts/mscoco2017.npz')['arr_0']
prompts = [str(p) for p in prompts[:10]]
prompts

['Athletes sitting on bench at sideline during game.',
 'A couple sits together on a bench next to the water. ',
 'The horses are all behind the barbwire fencing.',
 'A man with a polka dotted tie, dress shirt and wool sweater on.',
 'an elephant in a field of tall grass with trees in the background',
 'two people standing near one another playing nintendo wii',
 'Four bowls filled with different types of foods on top of a table.',
 'An image of boats, and ferries on water.',
 'A very  big kitchen with very bright colors and nice lighting.',
 'A gray train moving down the tracks with great speed.']

In [5]:
from solvers.euler_solver import Euler_Solver
from solvers.dpm_solver import DPM_Solver
from solvers.dual.dual_direct_dpm_solver import Dual_Direct_DPM_Solver

model_fn, noise_schedule, latents = model.get_model_fn(pos_conds=prompts, guidance_scale=4.5)

solver = Euler_Solver(model_fn, noise_schedule, algorithm_type='vector_prediction')
euler_latents = solver.sample(latents, steps=200, skip_type='time_uniform_flow', flow_shift=3.0)

  4%|▎         | 7/200 [00:04<02:25,  1.32it/s]

100%|██████████| 200/200 [02:34<00:00,  1.29it/s]


In [6]:
import torch.nn.functional as F

for NFE in [3, 5, 6, 8, 10, 15, 20, 25, 30]:
    solver = DPM_Solver(model_fn, noise_schedule, algorithm_type='data_prediction')
    dpm1_latents = solver.sample(latents, steps=NFE, order=2, skip_type="time_uniform_flow", method="multistep", flow_shift=3.0)
    
    solver = Dual_Direct_DPM_Solver(model_fn, noise_schedule, algorithm_type='dual_prediction')
    dpm2_latents = solver.sample(latents, steps=NFE, order=2, skip_type="time_uniform_flow", method="multistep", flow_shift=3.0)
    
    print('NFE :', NFE)
    print(F.mse_loss(euler_latents, dpm1_latents).item())
    print(F.mse_loss(euler_latents, dpm2_latents).item())

NFE : 3
0.8353796005249023
0.8995088934898376
NFE : 5
0.5819185972213745
0.6929311156272888
NFE : 6
0.4880281090736389
0.6629481911659241
NFE : 8
0.36159461736679077
0.5321329832077026
NFE : 10
0.27352744340896606
0.4360663592815399
NFE : 15
0.20011790096759796
0.2416311353445053
NFE : 20
0.0867912694811821
0.12411351501941681
NFE : 25
0.04295635223388672
0.03335348144173622
NFE : 30
0.032335031777620316
0.03713865205645561
